# Autores

- **Nome**: João Victor da Silva Batista de Farias **221022604**;
- **Nome**: Renan Vieira Guedes **221031363**;

**Vídeo de apresentação:** https://youtu.be/

# Jupyter Notebook

Código para deixar Widget do Jupyter Notebook condizente com o tema.

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

# Expectativas

O que a gente esperava do projeto, basicamente o que a gente queria que o modelo fosse capaz de fazer.

A motivação inicial do projeto foi tentarmos descobrir se com com um modelo relativamente pequeno, de até 100 milhões de parâmetros (por alguns motivos, como a falta de recursos de GPUs que custam muito caro), seria possível fazer ele entender a estrutura de problemas de programção competitiva e suas soluções e, a partir disso, ser capaz de generalizar para problemas que ele nunca viu antes.

Um dos maiores problemas que modelos de IA deste tamanho enfrentam é diferenciar a semântica da pragmática de uma estrutura de linguagem natural, até modelos maiores enfrentam isso, mas no nosso a gente sabia que não seria tão simples; então não tinhamos expectativas muito grandes, como gerar o código para um problema que ele nunca viu antes, muito menos dar de inicio a fim a solução para um problema de programação competitiva.

Então diminuimos o escopo e metas, e definimos como objetivo central fazer ele pelo menos pegar uma ideia principal para que ele podesse dar dicas de como resolver um problema de programação competitiva, como por exemplo, se ele consegue identificar que o problema é um problema de grafos, ou de programação dinâmica, ou de busca binária, etc.

Com isso, já poderia servir de ajuda para estudantes que estão entrando neste mundo, e até mesmo para programadores mais experientes que estão com dificuldades em resolver um problema com algum tipo de estrutura específica, como os de programação dinâmica, que nós mesmos tivemos dificuldades em reconhecer.

Então com tudo isso em mente, ao passar das últimas aulas de transformadores e LSTMs, nós fomos idealizando como seria nosso projeto.

# Modelo

Começamos carregando o dataset do arquivo "data.csv", que é o `Preprocessed_CompetitiveProgrammingDataset.csv` (feito para o Colab, se for local, só aponte para o caminho normal e renomei):
https://www.kaggle.com/datasets/dinuiongeorge/codeforces-competitive-programming-dataset

Note que o dataset é relativemente pequeno para treinamento (possui apenas 1500 exemplos), e infelizmente ele possui erros de parsing, por conta do Codeforces escrever a notação matemática em LaTeX, os raspadores de dados não conseguem pegar o texto completo, mas especificamente as variáveis e funções, então o dataset possui muitos exemplos com código incompleto; o que faz ele ser voltado ainda mais em estruturas de linguagem natural do que lógica de código. E isso mais pra frente, quando formos fazer a inferência, ficará mais claro exatamente o que queremos dizer aqui.

In [ ]:
#from google.colab import drive

#drive.mount('/content/drive')

DATA_PATH = './data.csv'

Pro Colab também, atualiza o pacote do Lightning pra gente utilizar com o Trainer.

In [ ]:
!pip install -U pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 63.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitli

Importar as bibliotecas necessárias para rodar o treinamento do modelo.

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

In [ ]:
import torch

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
import pytorch_lightning as pl

In [ ]:
import random

Essa função de pre-processamento vai ser usada para tokenizar os inputs e os targets,
ela vai adicionar o token "Problem:" antes de cada declaração de problema,
e vai usar o tokenizer em modo target para os labels.

O máximo de tokens que o t5-small aceita é 512, então vamos limitar o tamanho do input e do target.

A função `load_data` só carrega o arquivo csv e retorna a lista de tuplas, onde o primeiro item é o input e o segundo é o target.

E também processamos o dataset para adicionar o token "Problem:" e "Editorial" antes de cada problema e editorial, respectivamente.

Também adicionamos token de final do texto target.

In [ ]:
class CPProblemDataset(Dataset):
    def __init__(self, tokenizer, data_path, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = self.load_data(data_path)

    def load_data(self, path):
        data = []
        with open(path, "r") as f:
          for line in f:
            problem_statement, editorial = line.strip().split(",")

            data.append((problem_statement, editorial))

        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        problem, editorial = self.data[idx]

        # Pegar randomicamente o final do problema.
        if random.random() > 0.5:
            problem = problem[-692:]

        # Formatar o texto de entrada e saída.
        input_text = f"Problem: {problem}"
        output_text = f"Editorial: {editorial} {self.tokenizer.eos_token}"

        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        target_encoding = self.tokenizer(
            output_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }


Usamos inicialmente o modelo `t5-small` e o tokenizer `t5-small` para treinar com o nosso dataset porque ele não seria o suficente para aprender NLP e gerar textos coerentes, além de que, se tivessemos que treinar um modelo NLP com arquitetura transformador texto-para-texto, seriam usados muitas, muitas unidades de computação (ou seja, muito dinheiro), que não possuimos. Usar um modelo pré-treinado é uma forma de contornar isso e essêncial para que o projeto se conclua.

Quando finalizamos boa parte da arquiterua, escolhemos então o `ts-base` que tem 220 milhões de parâmetros, para ver se ele conseguiria aprender melhor a estrutura do dataset.

In [ ]:
MODEL_NAME = 't5-small'
#MODEL_NAME = 't5-base'
BATCH_SIZE = 8
MAX_LENGTH = 512

Modulo de dados que o Trainer do Transformer necessita para treinar o modelo.

Não traz nenhuma mudança em si muito grande além de definir o tamanho do batch, os trabalhadores persistentes e a randomização (que é importante, pois os dados estão ordenados por dificuldade de 4~5 itens como nas competições do Codeforces).

In [ ]:
class CPDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=BATCH_SIZE):
        super().__init__()
        self.batch_size = batch_size
        self.tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

    def setup(self, stage=None):
        self.dataset = CPProblemDataset(self.tokenizer, DATA_PATH, MAX_LENGTH)

    def train_dataloader(self):
        return DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4,
            persistent_workers=True
        )


In [ ]:
data_module = CPDataModule()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


No modelo, para penalizar certos problemas que tivemos, como repetição de tokens, halucinação muito cedo, tokens terminando mais cedo ou simplesmente do nada, tivemos de criar dois mecanismos de perda:
- `loss` é a perda padrão do modelo, que é a perda de cross-entropy entre o target e o output do modelo;
- adicionamos perda de fim de sequencia com `eos_token_id` para penalizar o modelo por terminar a sequência muito cedo;
- adicionamos `copy_penalty` para penalizar o modelo por repetir tokens dados pelo input.

Tinhamos adicionado outros, como o de repetição de tokens (`logits` e a função `calculate_repetions`), mas não se mostrou muito eficaz e acabou prejudicando o modelo, então comentamos eles.

Também definimos o Adam para otimizar o modelo, com uma taxa de aprendizado de 3e-4.

In [ ]:
# Módulo Lightning.
class CPModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
        self.tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

    def forward(self, input_ids, attention_mask, labels=None):
        return self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

    def training_step(self, batch, batch_idx):
        outputs = self(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['labels']
        )

        total_loss = outputs.loss

        # Penalizar se esquecer de adicionar o token de fim de sequência.
        # Porque o modelo estava gerando sequências infinitas.
        eos_positions = (batch['labels'] == self.tokenizer.eos_token_id).float()
        eos_loss = torch.mean(1 - eos_positions)
        # Peso da penalidade = 0.3
        total_loss += 0.3 * eos_loss

        # Inicial:
        #loss = output.loss
        #self.log('train_loss', loss)
        #return loss

        # Com sequência em gramas:
        # Calculcar overlap de tokens entre input e output.
        input_tokens = batch['input_ids']
        output_tokens = torch.argmax(outputs.logits, dim=-1)

        # Penalizar n-gramas iguais, podemos ajustar n = 3 para 2 ou 4
        copy_penalty = 0
        for seq_in, seq_out in zip(input_tokens, output_tokens):
            in_ngrams = set(tuple(seq_in[i:i+3]) for i in range(len(seq_in)-2))
            out_ngrams = set(tuple(seq_out[i:i+3]) for i in range(len(seq_out)-2))
            copy_penalty += len(in_ngrams & out_ngrams) / len(out_ngrams)

        # Quanto de peso adicionar a penalidade de cópia?
        # Neste caso 0.5, pois a penalidade de cópia é muito alta no nosso datset, que quase não repete tokens sequenciais.
        total_loss += 0.5 * copy_penalty
        self.log('train_loss', total_loss)
        return total_loss

        # Com logits:
        # logit(p) = log(p / (1 - p))
        #logits = outputs.logits
        #preds = torch.argmax(logits, dim=-1)

        # Calcularr repetição
        #rep_penalty = calculate_repetitions(preds) * 0.1

        #total_loss = outputs.loss + rep_penalty
        #self.log('train_loss', total_loss)


    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=3e-5)

Inicializar o modelo.

In [ ]:
model = CPModel()

Usamos:
- 10 épocas por questão do pequeno dataset que temos;
Tivemos muita repetição de tokens com épocas maiores, e o modelo não aprendeu direito com menos que isso.

- 0.5 de clipping para evitar gradientes muito grandes;
- '32-true' de precisão para usar 32 bits de precisão;
- já que a GPU falha localmente:
    - 2 de check_val_every_n_epoch para verificar o desempenho do modelo a cada 2 épocas;
    - 0.25 de val_check_interval para verificar o desempenho do modelo a cada 25% das épocas ;
    - 10 de log_every_n_steps para verificar o desempenho do modelo a cada 10 passos;

In [ ]:
trainer = pl.Trainer(
    max_epochs=10,
    gradient_clip_val=0.5,
    check_val_every_n_epoch=2,
    val_check_interval=0.25,
    log_every_n_steps=10,
    precision='32-true'
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M | eval
------------------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)
0         Modules in train mode
277       Modules in eval mode
/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  w

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


Salvamos o modelo para poder rodar localmente depois. ~~e não ter que ficar retreinando o mesmo modelo toda vez que o Google Colab reiniciar ou acabar o tempo de uso, já que somos pobres e não temos dinheiro para ficar comprando unidade computacional de GPU toda vez que precisamos treinar o modelo.~~

In [ ]:
model.model.save_pretrained("solver")
model.tokenizer.save_pretrained("solver")

('solver/tokenizer_config.json',
 'solver/special_tokens_map.json',
 'solver/spiece.model',
 'solver/added_tokens.json')

# Inferência

Definimos o nome do `device`, que se caso tivermos rodando localmente, vamos usar o CPU, e se tivermos rodando no Google Colab, vamos usar a GPU.

Usamos localmente os modelos treinados no Google Colab para tentarmos definir parâmetros melhores de inferência.

In [ ]:
device_name = torch.cuda.get_device_name(0)

torch_device_name = "cpu" if "AMD Radeon RX 580 2048SP" in device_name else "cuda"

print(f"Arquitetura de dispositivo: '{device_name}' e nome do dispositivo: '{torch_device_name}'" )

Arquitetura de dispositivo: 'NVIDIA A100-SXM4-40GB' e nome do dispositivo: 'cuda'


**Engenharia de prompt:**
Na função `solve` adicionamos preprocessamento do input para que o modelo possa entender melhor e gerar uma saída mais precisa.
Testamos vários tipos de strings de entrada, as menores tendem a criar saídas menos precisas.

Na de gerar a sequência, testamos vários, vários, vários tipos de parâmetros diferentes no intuito de criar algo que entregasse uma saída mais precisa e coerente, batendo com nossas expectativas.

In [ ]:
class CPSolver:
    def __init__(self, model_path="solver", torch_device_name=torch_device_name):
        self.model = T5ForConditionalGeneration.from_pretrained(model_path)
        self.tokenizer = T5Tokenizer.from_pretrained(model_path)
        self.device = torch.device(torch_device_name)
        self.model.to(self.device)

    def solve(self, problem_statement):
        # Preprocessamento do input para que o modelo possa entender melhor e gerar uma saída mais precisa.
        # Testamos vários tipos de strings de entrada, as menores tendem a criar saídas menos precisas.
        input_text = f"Generate the step-by-step programming for the problem: {problem_statement[:9000]}"

        inputs = self.tokenizer(
            input_text,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        ).to(self.device)

        outputs = self.model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=512,
            num_beams=4,
            early_stopping=False,
            length_penalty=-0.5,
            no_repeat_ngram_size=1,
            temperature=0.4,
            top_k=50,
            top_p=0.95,
            repetition_penalty=2.0,
            num_return_sequences=2,
            eos_token_id=self.tokenizer.eos_token_id,
            forced_eos_token_id=None

        )

        candidates = [self.tokenizer.decode(seq, skip_specials=True) for seq in outputs]
        return max(candidates, key=lambda x: len(x.split()))

In [ ]:
solver = CPSolver()

Algums problemas para testar com o modelo.

In [ ]:
problems = [
  "You are given two positive integers and In one move you can increase by replace with Your task is to find the minimum number of moves you need to do in order to make divisible by It is possible that you have to make moves as is already divisible by You have to answer independent test cases",
  "You have a matrix  filled with N integers. You want your matrix to become beautiful. The matrix is beautiful if the following two conditions are satisfied:  in each row, the first element is smaller than the second element;  in each column, the first element is smaller than the second element.   You can perform the following operation on the matrix any number of times: rotate it clockwise by  degrees, so the top left element shifts to the top right cell, the top right element shifts to the bottom right cell, and so on:  Determine if it is possible to make the matrix beautiful by applying zero or more operations.",
  "Polycarp has positive integers and He can perform the following operation Choose a integer and multiply of the integers or by Can Polycarp make it so that after performing the operation the sequence of three numbers forms an arithmetic progression Note that you the order of and Formally a sequence is called an arithmetic progression AP if there exists a number called common difference such that for all from to In this problem For example the following sequences are AP and The following sequences are not AP and You need to answer independent test cases ",
  "There are N pigeons numbered from 1 to N, and there are N nests numbered from 1 to N Initially, pigeon i is in nest i for 1 less than N You are given Q queries, which you must process in order. There are two types of queries, each given in one of the following formats: Move P pigeon to nest H, Output the number of nests that contain more than one pigeon.",
  "Adilbek was assigned to a special project For Adilbek it means that he has days to run a special program and provide its results But there is a problem the program needs to run for days to calculate the results Fortunately Adilbek can optimize the program If he spends is a non negative integer days optimizing the program he will make the program run in days is the ceiling function The program cannot be run and optimized simultaneously so the total number of days he will spend is equal to Will Adilbek be able to provide the generated results in no more than days "
]

In [ ]:
problem = problems[3]
solution = solver.solve(problem)

In [ ]:
import re

In [ ]:
def format_str(s):
    return re.sub(r'(?=[A-Z])', '\n', s)

In [ ]:
print("STATEMENT:")
print(format_str(problem))
print()
print("EDITORIAL GERADO:")
print(format_str(solution))

STATEMENT:

There are 
N pigeons numbered from 1 to 
N, and there are 
N nests numbered from 1 to 
N 
Initially, pigeon i is in nest i for 1 less than 
N 
You are given 
Q queries, which you must process in order. 
There are two types of queries, each given in one of the following formats: 
Move 
P pigeon to nest 
H, 
Output the number of nests that contain more than one pigeon.

EDITORIAL GERADO:
<pad> <extra_id_0>, and there are 
N pigeons that contain more than <extra_id_1> of the answer <extra_id_2> to nest 
H <extra_id_3> for this problem 
You can also use 
Q queries <extra_id_4> 
For example <extra_id_5> : <extra_id_6> by step programming <extra_id_7> 
Then you should check <extra_id_8> in <extra_id_9> from 1 
To do it we need <extra_id_10> with 
Nest number <extra_id_11> as well <extra_id_12> 
If is not greater then 
I would recommend doing so. 
In other words 
Let us know what happens when 
We don’t have any numbers on our list 
Of course they will be at least one <extra_id_13>

# Conclusões

Conseguimos alcançar mais ou menos o objetivo principal do projeto, que era fazer o modelo conseguir dar dicas de como resolver um problema de programação competitiva, como por exemplo, se ele consegue identificar que o problema é um problema de grafos, ou de programação dinâmica, ou de busca binária, etc.

Mas não temos tanta certeza se ele consegue entender a estrutura de problemas de programação competitiva, pois em alguns ele acaba halucinando e gerando uma saída que faz sentido até certo ponto, mas que possui algumas informações incorretas. O processo de inferência ajuda, mas requer uma engenharia de prompt mais forte para diferentes tamanhos de inputs, e até para inputs que eram para possuir imagens e notações em LaTeX.

Enfim, no futuro se conseguirmos mais dados e mais recursos computacionais, podemos tentar treinar um modelo maior e mais robusto, pois percebemos que o maior fator dele não conseguir generalizar tão bem aparenta ter sido a falta de dados, e não a arquitetura do modelo.

# Referências

Dinu Ion George;
Dataset do Codeforces de Programação Competitiva;
Acessado em 19 de fevereiro de 2025;
\<https://www.kaggle.com/datasets/dinuiongeorge/codeforces-competitive-programming-dataset\>

Google;
T5-Base;
Acessado em 19 de fevereiro de 2025;
\<https://huggingface.co/google-t5/t5-base\>

Google;
T5-Small;
Acessado em 19 de fevereiro de 2025;
\<https://huggingface.co/google-t5/t5-small\>

IBM;
What is NLP (natural language processing)?
Acessado em 19 de fevereiro de 2025;
\<https://www.ibm.com/think/topics/natural-language-processing\>

Restack;
Pytorch Lightning Gradient Clip Val;
Acessado em 19 de fevereiro de 2025;
\<https://www.restack.io/p/pytorch-lightning-answer-gradient-clip-val-cat-ai\>